# Agent2Agent Protocol (A2A) 

[![Agent2Agent Protocol (A2A)](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mozilla-ai/any-agent/blob/main/docs/cookbook/a2a.ipynb) 

Multi-Agent systems are complicated! Enter the [A2A](https://google-a2a.github.io/A2A) protocol by Google: this protocol allows for simpler communication between agents, and easily enables the use of an agent as a tool for another agent. 

In this tutorial we'll show how you can create a few agents with any-agent and have an agent be provided as a tool for the other agent.

This tutorial assumes basic familiarity with any-agent: if you haven't used any-agent before you may also find the [Creating your first agent](./../your_first_agent) cookbook to be useful.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [1]:
%pip install 'any-agent[a2a]'

import nest_asyncio

nest_asyncio.apply()

/home/daavoo/any-agent/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from getpass import getpass

# This notebook communicates with OpenAI GPT models using the OpenAI API.
if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

OPENAI_API_KEY found in environment.


## Configure the first two agents and serve them over A2A

Let's give our first two "helper" agents some very simple capabilities. 

For this demo, we'll use the async method `agent.serve_async` so that we can easily run all of the agents from inside the notebook in a single python process. 

In [3]:
from any_agent import AgentConfig, AnyAgent
from any_agent.serving import A2AServingConfig

/home/daavoo/any-agent/.venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/daavoo/any-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Time Agent

In [4]:
from datetime import datetime


def get_datetime() -> str:
    """Return the current date and time."""
    return str(datetime.now())


time_agent = await AnyAgent.create_async(
    "openai",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="gpt-4.1-nano",
        name="time_agent",
        description="Can help with getting the current date and time",
        tools=[get_datetime],
    ),
)

(time_task, time_server) = await time_agent.serve_async(
    A2AServingConfig(port=5000, endpoint="/time")
)

### Ski Expert Agent

In [5]:
from any_agent.tools import search_web

ski_expert_agent = await AnyAgent.create_async(
    "google",
    AgentConfig(
        model_id="gpt-4.1-nano",
        name="ski_expert_agent",
        instructions="You're an expert that is an avid skier, recommend a great location to ski given a time of the year",
        description="Can recommend my favorite location given a time of the year",
        tools=[search_web],
    ),
)

(ski_expert_task, ski_expert_server) = await ski_expert_agent.serve_async(
    A2AServingConfig(port=5001, endpoint="/ski_expert")
)

## Configure and use the Third Agent 

Now that the first two agents are serving over A2A, our main agent can be given access to use it just like a tool! In order to do this you use the `a2a_tool_async` function which retrieves the info about the agent and allows you to call it. 

In [6]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import a2a_tool_async

config = AgentConfig(
    model_id="gpt-4.1-mini",
    instructions="Use the available tools to obtain additional information to answer the query.",
    description="The orchestrator that can use other agents via tools using the A2A protocol.",
    tools=[
        await a2a_tool_async(
            "http://localhost:5000/time",
            http_kwargs={
                "timeout": 10
            },  # This gives the time agent up to 10 seconds to respond to each request
        ),
        await a2a_tool_async(
            "http://localhost:5001/ski_expert",
            http_kwargs={"timeout": 10},
        ),
    ],
)

agent = await AnyAgent.create_async(
    agent_framework="agno",
    agent_config=config,
)

agent_trace = await agent.run_async("Where should I go this weekend to ski?")

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "<instructions>\nUse the available tools to obtain additional information to answer the quer │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "Where should I go this weekend to ski?"                                                     │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "call_time_agent",                                                                         │ │
│ │     "tool.args": "{\"query\":\"current date\"}"                                                             │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 176,                                                                                      │ │
│ │   "output_tokens": 16                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "current date"                                                                               │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_datetime",                                                                            │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 40,                                                                                       │ │
│ │   "output_tokens": 10                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: get_datetime ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ 2025-06-10 13:40:56.692081                                                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The current date is June 10, 2025.                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 73,                                                                                       │ │
│ │   "output_tokens": 13                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── EXECUTE_TOOL: call_time_agent ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "current date"                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "id": "<function uuid4 at 0x7109b290c040>",                                                               │ │
│ │   "jsonrpc": "2.0",                                                                                         │ │
│ │   "result": {                                                                                               │ │
│ │     "contextId": null,                                                                                      │ │
│ │     "extensions": null,                                                                                     │ │
│ │     "kind": "message",                                                                                      │ │
│ │     "messageId": "21d190fb-f86c-4089-b708-2b3a77091627",                                                    │ │
│ │     "metadata": null,                                                                                       │ │
│ │     "parts": [                                                                                              │ │
│ │       {                                                                                                     │ │
│ │         "kind": "text",                                                                                     │ │
│ │         "metadata": null,                                                                                   │ │
│ │         "text": "The current date is June 10, 2025."                                                        │ │
│ │       }                                                                                                     │ │
│ │     ],                                                                                                      │ │
│ │     "referenceTaskIds": null,                                                                               │ │
│ │     "role": "agent",                                                                                        │ │
│ │     "taskId": null                                                                                          │ │
│ │   }                                                                                                         │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "You're an expert that is an avid skier, recommend a great location to ski given a time of t │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "best ski destination for June"                                                              │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_web",                                                                              │ │
│ │     "tool.args": {                                                                                          │ │
│ │       "query": "best ski destinations for June"                                                             │ │
│ │     }                                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 122,                                                                                      │ │
│ │   "output_tokens": 18                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── EXECUTE_TOOL: search_web ────────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "best ski destinations for June"                                                                 │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ ]8;id=661491;https://www.ski-lifts.com/guides/skiing-in-june-where-to-ski-in-june-july-beyond/\Skiing in June: Where to Ski in June, July & Beyond]8;;\ Tignes is one of France's best destinations for skiing  │ │
│ │ in June and skiing in July, thanks to the Grande Motte Glacier, which typically stays open until early      │ │
│ │ August. The glacier offers about 20km of slopes for all levels, from gentle blues to challenging reds, and  │ │
│ │ a popular summer snowpark for freestyle skiers and snowboarders. ]8;id=31702;https://localfreshies.com/snow-addict-2/\6 Ski Resorts Where You Can Go Skiing In ]8;;\  │ │
│ │ ]8;id=58872;https://localfreshies.com/snow-addict-2/\June & Possibly July]8;;\ Arapahoe Basin Average Prior Closing Date: Early June Image appears courtesy: Arapahoe │ │
│ │ Basin. A-Basin is #1 on our list for the best April skiing.This is because it's less about the corn and     │ │
│ │ more about skiing and snowboarding their packed powder. For the same reasons, the snow stays so good late   │ │
│ │ in the year, they can push the envelope to stay open. ]8;id=441813;https://www.onthesnow.com/news/the-best-ski-towns-in-summer/\The Best Ski Towns to Visit in Summer - OnTheSnow]8;;\ The │ │
│ │ Best Ski Towns To Visit In Summer Aspen, Colorado. Aspen is no doubt one of North America's best ski        │ │
│ │ destinations, and ranks near the top of many lists of the best ski towns in both winter and summer.         │ │
│ │ Cultural events and festivals are staples come summer in Aspen. Among the headliners is the Aspen Food and  │ │
│ │ Wine Classic each June. ]8;id=355074;https://www.powder.com/ski-resorts/best-ski-towns-summer\The 7 Best Ski Towns To Visit in the Summer - powder.com]8;;\ 2. Lake Tahoe, California. │ │
│ │ Lake Tahoe, California, is a bucket-list-worthy ski spot in the winter. Fifteen ski areas and ski resorts   │ │
│ │ are located around the lake, including the likes of Palisades Tahoe, Kirkwood, and Heavenly.The sheer       │ │
│ │ density of skiable terrain goes further with easily accessible backcountry zones. ]8;id=107179;https://www.travelandleisure.com/hotels-resorts/mountain-ski-resorts/us-ski-destinations-to-visit-in-summer\10 U.S. Ski Destinations ]8;;\ │ │
│ │ ]8;id=303752;https://www.travelandleisure.com/hotels-resorts/mountain-ski-resorts/us-ski-destinations-to-visit-in-summer\That Are Just As Spectacular in the Summer]8;;\ Here are 10 dynamic ski destinations in the U.S. that are        │ │
│ │ perfect for summer adventures, too. Even better, many offer lodging prices well below peak season rates.    │ │
│ │ Jackson Hole, Wyoming ]8;id=657072;https://travel.usnews.com/rankings/best-ski-destinations/\15 Best Ski Vacations in the World | U.S. News Travel]8;;\ #13 in Best Ski Vacations in    │ │
│ │ the World Located within Mont-Tremblant National Park in Canada's Quebec province, Mont-Tremblant Resort    │ │
│ │ attracts powder hounds with 100-plus runs spread across 755 acres. ]8;id=768565;https://powderquest.com/summer-skiing-destinations-top-7-ski-resorts-for-powder-skiing-in-the-summer/\Summer Skiing Destinations: 7 Ski ]8;;\       │ │
│ │ ]8;id=197

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ For skiing in June, some of the best destinations include:                                                  │ │
│ │                                                                                                             │ │
│ │  • Tignes, France: Thanks to the Grande Motte Glacier, it offers around 20 km of slopes and stays open      │ │
│ │    until early August, suitable for all levels.                                                             │ │
│ │  • Arapahoe Basin, Colorado, USA: Known for its late-season snow and packed powder, often stays open into   │ │
│ │    June.                                                                                                    │ │
│ │  • Lake Tahoe, California/Nevada, USA: While primarily a summer destination, nearby ski resorts like        │ │
│ │    Palisades Tahoe and Heavenly offer summer skiing opportunities.                                          │ │
│ │  • Mont-Tremblant, Canada: Features extensive skiing options and often extends into June with the right     │ │
│ │    conditions.                                                                                              │ │
│ │  • Tres Valles, Chile: In the Southern Hemisphere, it's perfect for winter skiing from June onwards.        │ │
│ │  • Iceland and Austria: Offer glacier skiing options that extend into June with the right conditions.       │ │
│ │                                                                                                             │ │
│ │ Would you like me to recommend a specific location based on your preferences such as terrain, ski style, or │ │
│ │ travel location?                                                                                            │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 1273,                                                                                     │ │
│ │   "output_tokens": 199                                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── EXECUTE_TOOL: call_ski_expert_agent ──────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "best ski destination for June"                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "id": "<function uuid4 at 0x7109b290c040>",                                                               │ │
│ │   "jsonrpc": "2.0",                                                                                         │ │
│ │   "result": {                                                                                               │ │
│ │     "contextId": null,                                                                                      │ │
│ │     "extensions": null,                                                                                     │ │
│ │     "kind": "message",                                                                                      │ │
│ │     "messageId": "9c1f744a-7b5b-4355-82ed-c8e5284c804f",                                                    │ │
│ │     "metadata": null,                                                                                       │ │
│ │     "parts": [                                                                                              │ │
│ │       {                                                                                                     │ │
│ │         "kind": "text",                                                                                     │ │
│ │         "metadata": null,                                                                                   │ │
│ │         "text": "For skiing in June, some of the best destinations include:\n\n- **Tignes, France:** Thanks │ │
│ │       }                                                                                                     │ │
│ │     ],                                                                                                      │ │
│ │     "referenceTaskIds": null,                                                                               │ │
│ │     "role": "agent",                                                                                        │ │
│ │     "taskId": null                                                                                          │ │
│ │   }                                                                                                         │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ For skiing this weekend in June, some of the best destinations include:                                     │ │
│ │                                                                                                             │ │
│ │  • Tignes, France (Grande Motte Glacier, open until early August)                                           │ │
│ │  • Arapahoe Basin, Colorado, USA (late-season snow, often open into June)                                   │ │
│ │  • Lake Tahoe, California/Nevada, USA (summer skiing opportunities in nearby resorts)                       │ │
│ │  • Mont-Tremblant, Canada (can extend into June with right conditions)                                      │ │
│ │  • Tres Valles, Chile (Southern Hemisphere winter skiing starting in June)                                  │ │
│ │  • Glacier skiing options in Iceland and Austria                                                            │ │
│ │                                                                                                             │ │
│ │ Would you like a specific recommendation based on your preferences such as terrain, ski style, or travel    │ │
│ │ location?                                                                                                   │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 657,                                                                                      │ │
│ │   "output_tokens": 131                                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Shut down the servers and the agents

The following code can be used to gracefully shut down the agents that are serving via A2A.

In [7]:
import asyncio

time_server.should_exit = True
while True:
    if time_task.done():
        print("Agent server task has completed!")
        break
    else:
        print("Waiting for server to exit")
        await asyncio.sleep(1)

ski_expert_server.should_exit = True
while True:
    if ski_expert_task.done():
        print("Agent server task has completed!")
        break
    else:
        print("Waiting for server to exit")
        await asyncio.sleep(1)

Waiting for server to exit
Agent server task has completed!
Waiting for server to exit
Agent server task has completed!
